In [8]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import to_categorical
from konlpy.tag import Okt  

In [2]:
res = pd.read_csv('resume.csv')
res = res.sort_values('resume_seq')
res = res.set_index(['resume_seq'])
res

,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code
resume_seq,,,,,,,,,,,,
U00001,2019-12-27,2020-02-01,4,2009,1500.0,1500.0,디자이너,재료·화학·섬유·의복,NaN,NaN,0,NaN
U00002,2020-04-24,2020-04-29,4,2005,0.0,0.0,디자이너,재료·화학·섬유·의복,NaN,NaN,73,기타 공공;개인서비스
U00003,2018-02-14,2020-07-08,4,2004,0.0,0.0,남성복디자이너;TD캐주얼,재료·화학·섬유·의복,NaN,NaN,113,섬유;봉제;가방;의류
U00004,2017-10-26,2020-04-27,4,2011,0.0,0.0,상품기획;영업기획,재료·화학·섬유·의복,NaN,NaN,27,섬유;봉제;가방;의류
U00005,2020-03-18,2020-04-08,4,2011,0.0,0.0,인사;총무;경영;MD;상품기획,재료·화학·섬유·의복,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,2020-02-05,2020-03-31,4,2003,0.0,2700.0,상품기획,재료·화학·섬유·의복,NaN,NaN,60,섬유;봉제;가방;의류
U08479,2016-02-17,2020-07-15,3,0,2500.0,2300.0,디자이너,재료·화학·섬유·의복,NaN,NaN,59,디자인;CAD;설계
U08480,2019-11-13,2020-06-24,4,0,0.0,0.0,VMD;텍스타일디자인;섬유디자인;니트디자인;여성복디자인,디자인,NaN,NaN,0,NaN


In [3]:
res_mer = res.drop(['reg_date','updated_date','job_code_seq2','job_code_seq3'],axis=1)
res_mer

,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,career_month,career_job_code
resume_seq,,,,,,,,
U00001,4,2009,1500.0,1500.0,디자이너,재료·화학·섬유·의복,0,NaN
U00002,4,2005,0.0,0.0,디자이너,재료·화학·섬유·의복,73,기타 공공;개인서비스
U00003,4,2004,0.0,0.0,남성복디자이너;TD캐주얼,재료·화학·섬유·의복,113,섬유;봉제;가방;의류
U00004,4,2011,0.0,0.0,상품기획;영업기획,재료·화학·섬유·의복,27,섬유;봉제;가방;의류
U00005,4,2011,0.0,0.0,인사;총무;경영;MD;상품기획,재료·화학·섬유·의복,0,NaN
...,...,...,...,...,...,...,...,...
U08478,4,2003,0.0,2700.0,상품기획,재료·화학·섬유·의복,60,섬유;봉제;가방;의류
U08479,3,0,2500.0,2300.0,디자이너,재료·화학·섬유·의복,59,디자인;CAD;설계
U08480,4,0,0.0,0.0,VMD;텍스타일디자인;섬유디자인;니트디자인;여성복디자인,디자인,0,NaN


In [4]:
res_mer.info()
for i in range(len(res_mer)):
    if res_mer["hope_salary"].iloc[i] == 0.:
        if res_mer["last_salary"].iloc[i] !=0.:
            res_mer["hope_salary"].iloc[i] = res_mer["last_salary"].iloc[i]
    elif res_mer["last_salary"].iloc[i] == 0.:
        if res_mer["hope_salary"].iloc[i] !=0.:
            res_mer["last_salary"].iloc[i] = res_mer["hope_salary"].iloc[i]    

<class 'pandas.core.frame.DataFrame'>
Index: 8482 entries, U00001 to U08482
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   degree           8482 non-null   int64  
 1   graduate_date    8482 non-null   int64  
 2   hope_salary      8482 non-null   float64
 3   last_salary      8482 non-null   float64
 4   text_keyword     8287 non-null   object 
 5   job_code_seq1    8482 non-null   object 
 6   career_month     8482 non-null   int64  
 7   career_job_code  7724 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 596.4+ KB


C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_mer["hope_salary"].iloc[i] = res_mer["last_salary"].iloc[i]
C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_mer["hope_salary"].iloc[i] = res_mer["last_salary"].iloc[i]
C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_mer["hope_salary"].iloc[i] = res_mer["last_salary"].iloc[i]
C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_mer["hope_salary"].iloc[i] = res_mer["last_salary"].iloc[i]
C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_mer["hope_salary"].iloc[i] = res_mer["last_salary"].iloc[i]
C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_mer["hope_salary"].iloc[i] = res_mer["last_salary"].iloc[i]
C:\Users\user\AppData\Local\Temp\ipykernel_7756\572712606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [5]:
res_mer['text_box'] = res_mer['text_keyword'].str.split(';')               # 텍스트 키워드 확인 
set_in = []

for i in res_mer['text_box']:
    if i is not None:
        if not isinstance(i, float):
            set_in.extend(i)

check = list(set(set_in))

In [6]:
check = [word.replace('MD', '기획') for word in check]        # MD라는 말을 기획으로 바꾸기

In [9]:
okt = Okt()
set_index = []
for i in check:
    setting = okt.nouns(i)
    set_index.extend(setting)
check = set_index

In [10]:
check_vc = pd.Series(check).value_counts()
check_vc2 = []
for i in check_vc.index:
    if len(i) > 1:
        check_vc2.append(i)
check_vc2

['기획',
 '영업',
 '관리',
 '디자인',
 '디자이너',
 '패션',
 '상품',
 '의류',
 '매장',
 '생산',
 '브랜드',
 '성복',
 '백화점',
 '경력',
 '해외',
 '남성',
 '대리점',
 '주얼',
 '쥬얼',
 '유통',
 '마케팅',
 '여성',
 '개설',
 '총괄',
 '개발',
 '스포츠',
 '구매',
 '우븐',
 '웨어',
 '바잉',
 '분석',
 '홍보',
 '스타일',
 '의상',
 '업무',
 '무역',
 '수입',
 '니트',
 '잡화',
 '소재',
 '바이어',
 '사업',
 '가능',
 '디스플레이',
 '지원',
 '신규',
 '광고',
 '아웃도어',
 '상담',
 '아울렛',
 '온라인',
 '용품',
 '골프',
 '부자',
 '할인점',
 '캐릭터',
 '신입',
 '쇼핑몰',
 '프로모션',
 '판매',
 '다이마루',
 '자재',
 '패션디자이너',
 '피팅',
 '엠디',
 '중국',
 '운영',
 '아동복',
 '컬러',
 '리스트',
 '유아',
 '로드',
 '원단',
 '담당',
 '그래픽',
 '데님',
 '정보',
 '편집',
 '교육',
 '트렌드',
 '국내',
 '샘플',
 '수출입',
 '섬유',
 '물류',
 '사무',
 '오픈',
 '핸들링',
 '핸드백',
 '인사',
 '가방',
 '상권',
 '경영',
 '인터넷',
 '실장',
 '관련',
 '수출',
 '품질',
 '유니섹스',
 '고객',
 '전략',
 '진행',
 '패턴',
 '인턴',
 '제작',
 '일반',
 '일러스트',
 '회계',
 '인재',
 '컨설팅',
 '디렉터',
 '자재관리',
 '입출',
 '경험',
 '런칭',
 '전산',
 '티셔츠',
 '막내',
 '악세사리',
 '부장',
 '신발',
 '업체',
 '아동',
 '정장',
 '발주',
 '가두',
 '제휴',
 '중국어',
 '행사',
 '매니저',
 '특종',
 '가죽',
 '캐주얼웨어',
 '전반',
 '관리자',
 '

In [11]:
box_index = (check_vc.index) 
for k in check_vc2:
    res_mer[f'{k}'] = 0  # 초기값 설정
    
for i in range(len(res_mer)):
    j = res_mer['text_keyword'][i]
    for k in range(len(check_vc2)):
        if isinstance(k, str):
            if re.search(check_vc2[k], j) is not None :
                res_mer[f'{k}'][i] = 1

C:\Users\user\AppData\Local\Temp\ipykernel_7756\123111402.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_mer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_7756\123111402.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_mer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_7756\123111402.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

C:\Users\user\AppData\Local\Temp\ipykernel_7756\123111402.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_mer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_7756\123111402.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_mer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_7756\123111402.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [12]:
scaler1 = StandardScaler()
res_mer['career_month'] = scaler1.fit_transform(res_mer['career_month'].values.reshape(-1, 1))

In [13]:
res_mer['degree']=res_mer['degree'].astype('category')                   # 5개의 유니크
res_mer['graduate_date']=res_mer['graduate_date'].astype('category')     # 49개의 유니크
res_mer['job_code_seq1']=res_mer['job_code_seq1'].astype('category')     # 3개의 유니크
res_mer['career_job_code']=res_mer['career_job_code'].astype('category') # 41개의 유니크

In [14]:
add1 = pd.get_dummies(res_mer['degree'],prefix ='degree',dtype='int') 
add2 = pd.get_dummies(res_mer['graduate_date'],prefix ='graduate',dtype='int') 
add3 = pd.get_dummies(res_mer['job_code_seq1'],prefix ='job_code',dtype='int') 
add4 = pd.get_dummies(res_mer['career_job_code'],prefix ='career_job',dtype='int') 
res_mer = pd.concat([res_mer,add1,add2,add3,add4],axis=1)

In [15]:
res_mer1=res_mer.drop(['degree','graduate_date','hope_salary','last_salary','job_code_seq1','career_job_code','text_box','text_keyword'],axis=1)

In [16]:
res_mer1

,career_month,기획,영업,관리,디자인,디자이너,패션,상품,의류,매장,...,career_job_인터넷서비스;포털;컨텐츠,career_job_자동차;운송장비;조선;항공,career_job_전기;가스;수도;에너지,career_job_전기;전자;통신장비,career_job_철강;금속;비금속;재료,career_job_출판;인쇄;편집;영상,career_job_컴퓨터;사무기기;소비재,career_job_학교;학원;직원훈련(교육서비스),career_job_호텔;숙박;관광;여행;외식,career_job_화학;고무;플라스틱
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,-1.092295,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00002,0.056512,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00003,0.685995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00004,-0.667393,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00005,-1.092295,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,-0.148070,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U08479,-0.163807,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U08480,-1.092295,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
